In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
import numpy as np
import torch as th
from syft import VirtualMachine
from pathlib import Path
from torchvision import datasets, transforms
from syft.core.plan.plan_builder import PLAN_BUILDER_VM, make_plan, build_plan_inputs, ROOT_CLIENT
from syft.lib.python.list import List
from matplotlib import pyplot as plt
from syft import logger
from syft import SyModule
logger.remove()

In [3]:
# Dataset
mnist_path = Path.home() / ".pysyft" / "mnist"
mnist_path.mkdir(exist_ok=True, parents=True)

mnist_train = datasets.MNIST(str(mnist_path), train=True, download=True,
               transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))

mnist_test = datasets.MNIST((mnist_path), train=False, 
              transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))

train_loader = th.utils.data.DataLoader(mnist_train, batch_size=32, shuffle=True, pin_memory=True)
test_loader = th.utils.data.DataLoader(mnist_test, batch_size=1024, shuffle=False, pin_memory=True)

# define model


In [4]:
class MySyModuleBlock(SyModule):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.p1 = th.nn.Parameter(th.rand(100,10) * 0.01)
    
    def forward(self, x):
        o1 = x @ self.p1
        return o1

In [5]:
class MySyModule(SyModule):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.layer1 = th.nn.Linear(28*28,100)
        self.relu1 = th.nn.ReLU()
        self.layer2 = MySyModuleBlock(input_size=(32,100))
    
    def forward(self, x):
        x_reshaped = x.view(-1, 28 * 28)
        o1 = self.layer1(x_reshaped)
        a1 = self.relu1(o1)
        out = self.layer2(x=a1)[0]
        return out

# Train plan

In [6]:
model = MySyModule(input_size=(32,28*28))

In [8]:
dummy_dl = sy.lib.python.List([next(iter(train_loader))])

In [9]:
remote_torch = ROOT_CLIENT.torch

In [10]:
@make_plan
def train(dl = dummy_dl, model = model):
    optimizer = remote_torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0)

    for xy in dl:
        optimizer.zero_grad()
        x, y = xy[0], xy[1]
        out = model(x=x)[0]
        loss = remote_torch.nn.functional.cross_entropy(out, y)
        loss.backward()
        optimizer.step()
    
    return [model]

## Util

In [11]:
def test(test_loader, model):
    correct = []
    model.eval()

    for data, target in test_loader:        
        output = model(x=data)[0]
        _, pred = th.max(output, 1)
        correct.append(th.sum(np.squeeze(pred.eq(target.data))))
    acc = sum(correct) / len(test_loader.dataset)
    return acc

# Train

In [12]:
alice_client = VirtualMachine(name="alice").get_client()
train_ptr = train.send(alice_client)

In [13]:
for i, (x, y) in enumerate(train_loader):
    dl = [[x,y]]
    res_ptr  = train_ptr(dl=dl, model=model)
    model, = res_ptr.get()
    
    if i%10 == 0 and i!=0:
        print(f"Iter: {i} Test accuracy: {test(test_loader, model):.2F}", flush=True)
    if i>50:
        break

/Users/koen/workspace/PySyft/src/syft/lib/torch/uppercase_tensor.py:30: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad = getattr(obj, "grad", None)


Iter: 10 Test accuracy: 0.32
Iter: 20 Test accuracy: 0.63
Iter: 30 Test accuracy: 0.69
Iter: 40 Test accuracy: 0.78
Iter: 50 Test accuracy: 0.78
